In [ ]:
# stdlib
import os
from pathlib import Path

# syft absolute
import syft as sy
from syft.service.code.user_code import UserCode
from syft.service.user.user import User

# Login

In [ ]:
server = sy.orchestra.launch(
    name="test_upgradability",
    dev_mode=True,
    reset=True,
    port="auto",
)

client = server.login(email="info@openmined.org", password="changethis")

In [ ]:
# Check if this is a new server
migration_data = client.get_migration_data()

assert len(migration_data.store_objects[User]) == 1
assert UserCode not in migration_data.store_objects

# Load migration data

In [ ]:
migration_data_dir = Path(os.getenv("MIGRATION_DATA_DIR", "."))
blob_path = migration_data_dir / "migration.blob"
yaml_path = migration_data_dir / "migration.yaml"

print(f"Loading migration data from {str(blob_path.resolve())}")

In [ ]:
res = client.load_migration_data(blob_path)
assert isinstance(res, sy.SyftSuccess), res.message

In [ ]:
res

# Post migration tests

In [ ]:
assert len(client.users.get_all()) == 2

In [ ]:
client_ds = server.login(email="ds@openmined.org", password="pw")

In [ ]:
# syft absolute
from syft.client.api import APIRegistry

In [ ]:
APIRegistry.__api_registry__.keys()

In [ ]:
code = client.code.get_all()[0]

In [ ]:
code.status

In [ ]:
req1 = client.requests[0]
req2 = client_ds.requests[0]
assert req1.status.name == "APPROVED" and req2.status.name == "APPROVED"
assert isinstance(req1._repr_html_(), str)
assert isinstance(req2._repr_html_(), str)

In [ ]:
jobs = client_ds.jobs
assert isinstance(jobs[0]._repr_html_(), str)

In [ ]:
ds = client_ds.datasets
asset = ds[0].assets[0]

In [ ]:
res = client_ds.code.compute_mean(data=asset)

In [ ]:
assert res.get().shape == (100_000, 5)

In [ ]:
jobs = client_ds.jobs.get_all()

In [ ]:
job = jobs[0]

In [ ]:
job.logs()

In [ ]:
logs = job.logs(_print=False)
assert isinstance(logs, str)

In [ ]:
if server.server_type.value == "python":
    server.land()